In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)



In [2]:
import sys
if('/kaggle/input/references') not in sys.path:
    print(sys.path.append('/kaggle/input/references'))
    print('Adding cite path is done')
else:
    print('Cite path added')

None
Adding cite path is done


In [3]:
!pip install chardet
!pip install pycocotools -i https://pypi.tuna.tsinghua.edu.cn/simple/

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 7.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.8/427.8 kB 486.2 kB/s eta 0:00:00


In [4]:
from PIL import Image
import cv2
#bk_img = Book_Image('/kaggle/input/data-train/data_train/IMG_20191014_161641.json')
#img=np.array(bk_img.image)


def trans(image):
    img=np.array(image)
    # 形态学操作：膨胀，增强边缘
    kernel = np.ones((3, 3), np.uint8)
    img = cv2.dilate(img, kernel, iterations=1)


    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    

    # 步骤4: 边缘检测 - 使用Canny算法
    edges = cv2.Canny(gray_image, threshold1=50, threshold2=150)
    # 形态学开运算：先腐蚀再膨胀，用于断开边缘上的细节
    opened_edges = cv2.morphologyEx(edges, cv2.MORPH_OPEN, kernel)

    # 步骤5: 可选的轮廓处理
    # 寻找轮廓
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour_image = cv2.cvtColor(gray_image, cv2.COLOR_GRAY2BGR)
    cv2.drawContours(contour_image, contours, -1, (0, 255, 0), 2)

    # 绘制轮廓
    contour_img = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)
    cv2.drawContours(contour_img, contours, -1, (0, 255, 0), 2)
    #cv2.imwrite('/kaggle/working/blurred_image.jpg', img_blurred)
    #cv2.imwrite('/kaggle/working/edges.jpg', edges)
    #cv2.imwrite('/kaggle/working/contours.jpg', contour_img)
    #cv2.imwrite('/kaggle/working/contour_image.jpg', contour_image)
    return contour_image

In [5]:
import base64
import json
from copy import deepcopy
import chardet
import cv2
import numpy as np
import torch.utils.data
from PIL import Image, ImageDraw
import io
import os
from torchvision import tv_tensors

class Book_Image():
    def __init__(self,file_dir,with_mask=False,encode=r'ISO-8859-1'):
        self.dir=dir
        self.image_json=self.file_json(file_dir,encode)
        self.image,self.shapes_list,self.masks,self.masks_boxes=self.load_image(self.image_json,with_mask=with_mask)
        self.img_array=np.array(self.image)
        self.trans_image=trans(self.image)
    def file_json(self,file_dir,encode):
        with open(os.path.join(file_dir), 'r', encoding=encode) as f:
            image_json = json.loads(f.read())
        return image_json
    def load_image(self,image_json,with_mask=False):
        base64_img_data=image_json['imageData']
        img_data=base64.b64decode(base64_img_data)
        shapes_list = [shape['points'] for shape in image_json['shapes']]
        image=Image.open(io.BytesIO(img_data))
        
        masks=[]
        masks_boxes = []
        if with_mask == True:
            for m in image_json['mask_data']:
                m_data=base64.b64decode(m)
                m_=Image.open(io.BytesIO(m_data)).convert('1')
                masks.append(m_)
                pos=np.where(m_)    #返回值为1，或为true的像素点的坐标 pos[0]是y，pos[1]是x
                xmin=np.min(pos[1])
                xmax=np.max(pos[1])
                ymin=np.min(pos[0])
                ymax=np.max(pos[0])
                masks_boxes.append([xmin,ymin,xmax,ymax])
        return image,shapes_list,masks,masks_boxes
    def show(self):
        self.image.show()
    def draw_boxes(self):
        if self.image!=None:
            draw=ImageDraw.Draw(self.image)
            for s in self.shapes_list:
                p_count = len(s)
                for idx in range(p_count):
                    draw.line((s[idx % p_count][0], s[idx % p_count][1], s[(idx + 1) % p_count][0],
                               s[(idx + 1) % p_count][1]), fill=(255, 0, 0), width=5)
    def draw_masks(self):
        if self.image!=None:
            another_image=deepcopy(self.image)
            draw=ImageDraw.Draw(another_image)
            for s in self.shapes_list:
                #mask_array=np.zeros(self.img_array.shape[:2],dtype=np.uint8)    #np.uint8 无符号数
                polygon=[tuple(idx) for idx in s]
                draw.polygon(polygon,fill=(255,255,255),outline=None,width=0)
            return another_image
    def get_masks(self):
        if self.image!=None:
            mask_list=list()
            for s in self.shapes_list:
                # mask_array=np.zeros(self.img_array.shape[:2],dtype=np.uint8)    #np.uint8 无符号数
                mask = Image.new('L', self.image.size, color=0)
                draw = ImageDraw.Draw(mask)
                polygon = [tuple(idx) for idx in s]
                draw.polygon(polygon, fill=255, outline=None, width=0)
            #mask.convert('1').show()
                mask_list.append(mask)
            return mask_list

class Book_Spine_Dataset(torch.utils.data.Dataset):
    def __init__(self,root):
        self.root=root
        self.imgs= list(sorted(os.listdir(self.root)))
    def __getitem__(self, idx):
        bk_img = Book_Image(os.path.join(self.root, self.imgs[idx]), with_mask=True, encode='utf-8')

        # Wrap sample and targets into torchvision tv_tensors:
        image = torch.as_tensor(np.array(bk_img.trans_image, dtype=np.uint8), dtype=torch.float32)
        image = image.permute(2, 0, 1).div(255)

        num_objs = len(bk_img.masks)
        labels = torch.ones((num_objs,), dtype=torch.int64)
        m_boxes = torch.as_tensor(bk_img.masks_boxes, dtype=torch.float32)
        area = (m_boxes[:, 3] - m_boxes[:, 1]) * (m_boxes[:, 2] - m_boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.ones((num_objs,), dtype=torch.int64)

        target = {}
        target['boxes'] = m_boxes
        target['labels'] = labels
        target['masks'] = torch.as_tensor(
            np.array([np.array(m.convert('1'), dtype=np.uint8) for m in bk_img.masks], dtype=np.uint8),
            dtype=torch.uint8)
        target['image_id'] = self.imgs[idx]
        target['area'] = area
        target['iscrowd'] = iscrowd
        return image, target

    def __len__(self):
        return len(self.imgs)

In [6]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor


def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights="DEFAULT")

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(
        in_features_mask,
        hidden_layer,
        num_classes
    )

    return model

In [7]:
import utils
from engine import train_one_epoch,evaluate
from tqdm import tqdm
def start_train():
    torch.cuda.empty_cache()
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    model = get_model_instance_segmentation(2)
    #print(model)
    model.to(device)  ##将模型加载到处理器中
    dataset= Book_Spine_Dataset('/kaggle/input/data-train/data_train')

    ###将数据集切分为训练集和测试集
    indices = torch.randperm(len(dataset)).tolist()  ##生成随机数索引的列表
    dataset_for_train= torch.utils.data.Subset(dataset, indices[:120])  #根据随机索引切分子集    Subset(数据集,索引列表)
    dataset_for_test = torch.utils.data.Subset(dataset, indices[120:])

    ###data_loader用于训练时迭代加载数据集
    data_loader=torch.utils.data.DataLoader(dataset_for_train,batch_size=5,shuffle=True,collate_fn=utils.collate_fn)
    data_loader_test=torch.utils.data.DataLoader(dataset_for_test,batch_size=1,shuffle=False,collate_fn=utils.collate_fn)



    params=[p for p in model.parameters() if p.requires_grad]
    optimizer=torch.optim.SGD(params,lr=0.0003,momentum=0.5,weight_decay=0.0005)

    lr_scheduler=torch.optim.lr_scheduler.StepLR(optimizer,step_size=2,gamma=0.1)

    num_epochs=1

    for epoch in tqdm(range(num_epochs)):
        train_one_epoch(model,optimizer,data_loader,device,epoch,print_freq=5)
        model.eval()
        torch.save(model,f'/kaggle/working/bk_spine_segmentation{epoch}.pth')
        lr_scheduler.step()
        #evaluate(model,data_loader_test,device=device)
    print("That's it")
start_train()

Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/hub/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth
100%|██████████| 170M/170M [00:01<00:00, 161MB/s]
  0%|          | 0/1 [00:00<?, ?it/s]

Epoch: [0]  [ 0/24]  eta: 0:06:09  lr: 0.000013  loss: 4.2297 (4.2297)  loss_classifier: 0.8788 (0.8788)  loss_box_reg: 0.9098 (0.9098)  loss_mask: 1.7894 (1.7894)  loss_objectness: 0.2999 (0.2999)  loss_rpn_box_reg: 0.3519 (0.3519)  time: 15.3783  data: 12.7661  max mem: 7255
Epoch: [0]  [ 5/24]  eta: 0:04:37  lr: 0.000078  loss: 4.1836 (4.2329)  loss_classifier: 0.8542 (0.8564)  loss_box_reg: 0.8974 (0.8894)  loss_mask: 1.7894 (1.9965)  loss_objectness: 0.1224 (0.1806)  loss_rpn_box_reg: 0.3323 (0.3100)  time: 14.6294  data: 13.1181  max mem: 7759
Epoch: [0]  [10/24]  eta: 0:03:22  lr: 0.000144  loss: 3.5387 (3.7812)  loss_classifier: 0.8330 (0.8152)  loss_box_reg: 0.8956 (0.8846)  loss_mask: 1.3883 (1.6479)  loss_objectness: 0.0933 (0.1249)  loss_rpn_box_reg: 0.3274 (0.3086)  time: 14.4538  data: 13.0437  max mem: 7759
Epoch: [0]  [15/24]  eta: 0:02:10  lr: 0.000209  loss: 3.2815 (3.5091)  loss_classifier: 0.7715 (0.7719)  loss_box_reg: 0.8945 (0.8856)  loss_mask: 1.2663 (1.4550)  l

100%|██████████| 1/1 [05:48<00:00, 348.22s/it]

That's it


In [8]:
from copy import deepcopy
def just_a_try():
    bk_img=Book_Image('/kaggle/input/data-train/data_train/IMG_20191014_161649.json',with_mask=True,encode='utf-8')
    
    image = torch.as_tensor(np.array(bk_img.trans_image, dtype=np.uint8), dtype=torch.float32)
    image = image.permute(2, 0, 1).div(255)
        
    # Wrap sample and targets into torchvision tv_tensors:
    #print(image)
    
    torch.cuda.empty_cache()
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    
    model = torch.load('/kaggle/working/bk_spine_segmentation0.pth')
    model.to(device)
    model.eval()
    print('predicting')
    with torch.no_grad():
        output = model([image.to(device)])
        pred = output[0]
    
    masks = (pred["masks"] > 0.7).squeeze(1)
    print(pred)

    mm=masks[0].cpu().numpy()
    mi=Image.fromarray(mm,mode='L')
    mi.save('/kaggle/working/mask.jpeg',format='jpeg')
    print(mm)
    box=list(pred['boxes'][5].cpu().numpy())
    print(box)

    draw=ImageDraw.Draw(bk_img.image)
    draw.rectangle(box,fill=(255,255,255),outline=None,width=0)
    bk_img.image.save('/kaggle/working/toshow_image.jpeg',format='jpeg')
just_a_try()

predicting
{'boxes': tensor([[2.7579e+03, 0.0000e+00, 2.9261e+03, 2.1397e+03],
        [8.4804e+02, 1.6313e+03, 8.9802e+02, 1.7976e+03],
        [5.6703e+02, 0.0000e+00, 3.9382e+03, 4.1034e+02],
        [2.4802e+03, 0.0000e+00, 2.8317e+03, 3.0240e+03],
        [1.1522e+02, 7.3616e+01, 4.5195e+02, 3.0240e+03],
        [2.7781e+03, 3.2408e+00, 2.8996e+03, 1.2241e+03],
        [2.5681e+03, 0.0000e+00, 2.7485e+03, 1.9846e+03],
        [1.8620e+03, 1.0612e+01, 2.0067e+03, 3.0240e+03],
        [3.9114e+03, 2.6719e+03, 4.0053e+03, 3.0069e+03],
        [2.8230e+03, 0.0000e+00, 2.9217e+03, 1.3578e+03],
        [2.9524e+03, 0.0000e+00, 3.1997e+03, 3.0240e+03],
        [2.8293e+03, 2.1893e+01, 3.0071e+03, 2.3261e+03],
        [2.6302e+03, 1.4528e+02, 2.9821e+03, 3.0240e+03],
        [7.5507e+01, 1.5142e+02, 1.0745e+02, 1.8837e+02],
        [8.1970e+02, 1.6220e+03, 8.7364e+02, 1.8081e+03],
        [3.7222e+03, 3.8624e+01, 3.8852e+03, 2.6576e+03],
        [2.6776e+03, 9.2841e+00, 2.7704e+03, 1.1724